## Imports

In [1]:
from datetime import datetime as dt
import MetaTrader5 as mt5
from time import sleep
import pandas as pd
import pytz

## MT5 relevant info

In [2]:
# display data on the MetaTrader 5 package
print("MetaTrader5 package author: ",mt5.__author__)
print("MetaTrader5 package version: ",mt5.__version__)

MetaTrader5 package author:  MetaQuotes Software Corp.
MetaTrader5 package version:  5.0.33


## MT5 connect

In [3]:
# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()
else:
    print(mt5.initialize())

True


## Trading conditions

In [4]:
class Symbol:
    
    def __init__(self,
                 name="",
                 lot_size=0,
                 stop_loss=0,
                 take_profit=0,
                 dataframe_M15=pd.DataFrame(),
                 dataframe_M30=pd.DataFrame(),
                 dataframe_H1=pd.DataFrame(),
                 dataframe_H4=pd.DataFrame()):
        
        self.name = name
        self.lot_size = lot_size
        self.stop_loss = stop_loss
        self.take_profit = take_profit
        self.dataframe_M15 = dataframe_M15
        self.dataframe_M30 = dataframe_M30
        self.dataframe_H1 = dataframe_H1
        self.dataframe_H4 = dataframe_H4


In [5]:
Volatility_10 = Symbol(name="Volatility 10 Index",
                       lot_size=0.3,
                       stop_loss=4000,
                       take_profit=2000)

Volatility_10_1s = Symbol(name="Volatility 10 (1s) Index",
                          lot_size=0.2,
                          stop_loss=500,
                          take_profit=1000)

Volatility_25 = Symbol(name="Volatility 25 Index",
                       lot_size=0.5,
                       stop_loss=6000,
                       take_profit=3000)

Volatility_25_1s = Symbol(name="Volatility 25 (1s) Index",
                          lot_size=0.005,
                          stop_loss=45000,
                          take_profit=22000)

Volatility_50 = Symbol(name="Volatility 50 Index",
                       lot_size=3,
                       stop_loss=5000,
                       take_profit=2500)

Volatility_50_1s = Symbol(name="Volatility 50 (1s) Index",
                          lot_size=0.005,
                          stop_loss=60000,
                          take_profit=30000)

Volatility_75 = Symbol(name="Volatility 75 Index",
                       lot_size=0.001,
                       stop_loss=250000,
                       take_profit=100000)

Volatility_75_1s = Symbol(name="Volatility 75 (1s) Index",
                          lot_size=0.005,
                          stop_loss=4600,
                          take_profit=2300)

Volatility_100 = Symbol(name="Volatility 100 Index",
                        lot_size=0.2,
                        stop_loss=2000,
                        take_profit=1000)

Volatility_100_1s = Symbol(name="Volatility 100 (1s) Index",
                           lot_size=0.1,
                           stop_loss=4600,
                           take_profit=2300)

Step_index = Symbol(name="Step Index",
                    lot_size=0.1,
                    stop_loss=40,
                    take_profit=20)

Boom_500 = Symbol(name="Boom 500 Index",
                    lot_size=0.2,
                    stop_loss=50000,
                    take_profit=50000)

Boom_1000 = Symbol(name="Boom 1000 Index",
                    lot_size=0.2,
                    stop_loss=50000,
                    take_profit=50000)

Crash_500 = Symbol(name="Crash 500 Index",
                    lot_size=0.2,
                    stop_loss=50000,
                    take_profit=50000)

Crash_1000 = Symbol(name="Crash 1000 Index",
                    lot_size=0.2,
                    stop_loss=50000,
                    take_profit=50000)



In [6]:
SYMBOLS = [
           Volatility_10,
           Volatility_10_1s,
           Volatility_25,
           Volatility_25_1s,
           Volatility_50,
           Volatility_50_1s,
           Volatility_75,
           Volatility_75_1s,
           Volatility_100,
           Volatility_100_1s,
           Step_index,
           Crash_1000,
           Crash_500,
           Boom_1000,
           Boom_500
          ]

TIMEFRAMES = [
    mt5.TIMEFRAME_M15,
    mt5.TIMEFRAME_M30,
    mt5.TIMEFRAME_H1,
    mt5.TIMEFRAME_H4    
]


In [7]:
current_tick = float()
last_tick = float()

In [8]:
symbol_to_trade = Volatility_10_1s
timeframe = 60
close_order_deviation = 20
current_ea_comments = "Exoity V1.3"

## Defining tick, buy and sell functions

In [9]:
def get_last_tick(symbol):
    last_tick = mt5.symbol_info_tick(symbol)
    return last_tick

In [10]:
def send_order_sell(symbol = "Volatility 75 Index", comments = "python script open",lot_size=0.001,take_profit=250000,stop_loss=100000):
    # prepare the sell request structure
    symbol_info = mt5.symbol_info(symbol)
    
    if symbol_info is None:
        print(symbol, "not found, can not call order_check()")
        mt5.shutdown()
        quit()

    lot = lot_size
    STOP_LOSS = stop_loss
    TAKE_PROFIT = take_profit
    #lot = 1.5
    point = mt5.symbol_info(symbol).point
    price = mt5.symbol_info_tick(symbol).bid
    deviation = 20
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_SELL,
        "price": price,
        "sl": price + STOP_LOSS * point,
        "tp": price - TAKE_PROFIT * point,
        "deviation": deviation,
        "magic": 234000,
        "comment": comments,
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK,
    }

    # send a trading request
    result = mt5.order_send(request)
    
    return result, request

In [11]:
def send_order_buy(symbol = "Volatility 75 Index", comments = "python script open",lot_size=0.001,take_profit=250000,stop_loss=100000):
    # prepare the buy request structure
    symbol_info = mt5.symbol_info(symbol)
    
    if symbol_info is None:
        print(symbol, "not found, can not call order_check()")
        mt5.shutdown()
        quit()

    lot = lot_size
    STOP_LOSS = stop_loss
    TAKE_PROFIT = take_profit
    #lot = 1.5
    point = mt5.symbol_info(symbol).point
    price = mt5.symbol_info_tick(symbol).ask
    deviation = 20
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_BUY,
        "price": price,
        "sl": price - STOP_LOSS * point,
        "tp": price + TAKE_PROFIT * point,
        "deviation": deviation,
        "magic": 234000,
        "comment": comments,
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_FOK,
    }

    # send a trading request
    result = mt5.order_send(request)
    
    return result, request

In [12]:
def close_trade(action, buy_request, result, deviation):
    '''
        https://www.mql5.com/en/docs/integration/python_metatrader5/mt5ordersend_py
    '''
    # create a close request
    symbol = buy_request['symbol']
    if action == 'buy':
        trade_type = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).ask
    elif action =='sell':
        trade_type = mt5.ORDER_TYPE_SELL
        price = mt5.symbol_info_tick(symbol).bid
    position_id=result.order
    lot = buy_request['volume']

    close_request={
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": trade_type,
        "position": position_id,
        "price": price,
        "deviation": deviation,
        "magic": 12000,
        "comment": "python script close",
        "type_time": mt5.ORDER_TIME_GTC, # good till cancelled
        "type_filling": mt5.ORDER_FILLING_FOK,
    }
    # send a close request
    result = mt5.order_send(close_request)
    return result

## Testing the best symbol to run the strategy on

In [13]:
def determine_result(open_price,close_price):
    if open_price > close_price:
        return 'negative'
    elif open_price <= close_price:
        return 'positive'

In [14]:
date_from = dt(2019,1,1)
date_to = dt(2021,3,1)

In [15]:
for symbol in SYMBOLS:
    
    for timeframe in TIMEFRAMES: 
        
        if timeframe == mt5.TIMEFRAME_M15:
            rates = mt5.copy_rates_range(symbol.name, mt5.TIMEFRAME_M15, date_from, date_to)
            rates_frame = pd.DataFrame(rates)
            rates_frame = rates_frame.drop(['real_volume','time','spread','tick_volume'],axis=1)
            symbol.dataframe_M15 = rates_frame
            symbol.dataframe_M15["result"] = symbol.dataframe_M15.apply(lambda row: determine_result(row['open'],row['close']),axis=1)
        
        if timeframe == mt5.TIMEFRAME_M30:
            rates = mt5.copy_rates_range(symbol.name, mt5.TIMEFRAME_M30, date_from, date_to)
            rates_frame = pd.DataFrame(rates)
            rates_frame = rates_frame.drop(['real_volume','time','spread','tick_volume'],axis=1)
            symbol.dataframe_M30 = rates_frame
            symbol.dataframe_M30["result"] = symbol.dataframe_M30.apply(lambda row: determine_result(row['open'],row['close']),axis=1)
            
        if timeframe == mt5.TIMEFRAME_H1:
            rates = mt5.copy_rates_range(symbol.name, mt5.TIMEFRAME_H1, date_from, date_to)
            rates_frame = pd.DataFrame(rates)
            rates_frame = rates_frame.drop(['real_volume','time','spread','tick_volume'],axis=1)
            symbol.dataframe_H1 = rates_frame
            symbol.dataframe_H1["result"] = symbol.dataframe_H1.apply(lambda row: determine_result(row['open'],row['close']),axis=1)
            
        if timeframe == mt5.TIMEFRAME_H4:
            rates = mt5.copy_rates_range(symbol.name, mt5.TIMEFRAME_H4, date_from, date_to)
            rates_frame = pd.DataFrame(rates)
            rates_frame = rates_frame.drop(['real_volume','time','spread','tick_volume'],axis=1)
            symbol.dataframe_H4 = rates_frame
            symbol.dataframe_H4["result"] = symbol.dataframe_H4.apply(lambda row: determine_result(row['open'],row['close']),axis=1)

In [16]:
for symbol in SYMBOLS:
    timeframes = [
                    symbol.dataframe_M15,
                    symbol.dataframe_M30,
                    symbol.dataframe_H1,
                    symbol.dataframe_H4
                 ]
    names = [
                "M15",
                "M30",
                "H1",
                "H4"
            ]
    for timeframe, name in zip(timeframes,names):
        
        candles = timeframe['result'].tolist()

        win_rate = float()
        win_qty = float()


        for i in range(0, len(candles)):
            try:
                if candles[i] == candles[i+1]:
                    win_qty += 1
            except:
                pass 


        win_rate = win_qty / len(candles)
        if win_rate >= 0.5041:
            print("{} at {} => {}".format(symbol.name, name, win_rate))

Volatility 10 (1s) Index at H1 => 0.504574019568849
Volatility 50 (1s) Index at M30 => 0.5041894073333846
Crash 1000 Index at M15 => 0.5215092269478689
Crash 1000 Index at M30 => 0.5102389078498294
Crash 1000 Index at H1 => 0.5062157221206581
Crash 500 Index at M15 => 0.5102834036323073
Boom 1000 Index at M15 => 0.526507474513509
Boom 1000 Index at M30 => 0.5108483666504144
Boom 500 Index at M15 => 0.51188691741904


## Execute the cell below to run strategy

In [106]:
has_last_price = False
buy_signal = False
sell_signal = False

while True:


    if has_last_price == False and dt.now().minute % timeframe == 0:
        last_price = get_last_tick(symbol_to_trade.name).ask
        has_last_price = True

        sleep(180)

    if has_last_price == True and dt.now().minute % timeframe == 0:
        current_price = get_last_tick(symbol_to_trade.name).ask

        delta = current_price - last_price
        if delta > 0:
            buy_signal = True
        else:
            sell_signal = True

        break

    sleep(1)

In [107]:

if buy_signal == True:
    result, request = send_order_buy(symbol=symbol_to_trade.name,
                                     lot_size=symbol_to_trade.lot_size,
                                     stop_loss=symbol_to_trade.stop_loss,
                                     take_profit=symbol_to_trade.take_profit,
                                     comments=current_ea_comments)
elif sell_signal == True:
    result, request = send_order_sell(symbol=symbol_to_trade.name,
                                      lot_size=symbol_to_trade.lot_size,
                                      stop_loss=symbol_to_trade.stop_loss,
                                      take_profit=symbol_to_trade.take_profit,
                                      comments=current_ea_comments)

sleep(80)


In [ ]:
while True:
    buy_signal = False
    sell_signal = False
    if dt.now().minute % timeframe == 0:

        close_trade('sell',request,result,close_order_deviation)
        close_trade('buy',request,result,close_order_deviation)

        last_price = current_price
        current_price = get_last_tick(symbol_to_trade.name).ask
        delta = current_price - last_price

        if delta > 0:
            buy_signal = True
        elif delta < 0:
            sell_signal = True

        if buy_signal == True:
            result, request = send_order_buy(symbol=symbol_to_trade.name,
                                             lot_size=symbol_to_trade.lot_size,
                                             stop_loss=symbol_to_trade.stop_loss,
                                             take_profit=symbol_to_trade.take_profit,
                                             comments=current_ea_comments)
        elif sell_signal == True:
            result, request = send_order_sell(symbol=symbol_to_trade.name,
                                              lot_size=symbol_to_trade.lot_size,
                                              stop_loss=symbol_to_trade.stop_loss,
                                              take_profit=symbol_to_trade.take_profit,
                                              comments=current_ea_comments)

        sleep(120)

    sleep(1)
